Ensure you are using the preinstalled Jupyter kernel `amcinsights`

# Workflow Management Service

Set up a customer in the TPS Interface and authenticate with Amazon Ads Auth before using

#### -----------------------
## Global Configs
Run this cell to initialize the notebook interface

In [14]:
import json
import re
import boto3
import os
from dotenv import load_dotenv
load_dotenv()

invoke_workflow_lambda_name = os.environ['INVOKE_WORKFLOW_SM_NAME']
invoke_workflow_executions_lambda_name = os.environ['INVOKE_WORKFLOW_EXECUTION_SM_NAME']
create_workflow_schedule_lambda_name = os.environ['CREATE_WORKFLOW_SCHEDULE_NAME']
delete_workflow_schedule_lambda_name = os.environ['DELETE_WORKFLOW_SCHEDULE_NAME']

boto3_session = boto3.Session()
lambda_client = boto3_session.client('lambda')

#### -----------------------
## Customer Id
Edit the value below with the Customer Id you want to submit requests for

In [ ]:
CUSTOMER_ID = "democustomer"

#### -----------------------
# Workflow Requests
The examples below show how you can use the Workflow Management Service to interact with the application and Amazon Marketing Cloud API.

For each request type, the top cell shows the parameters you can update while the bottom cell will run the request and return a link.

### Create Workflow

In [ ]:
workflow_definition = {
    "workflowId": "test-wfm-1",
    "filteredMetricsDiscriminatorColumn": "filtered",
    "inputParameters": [
        {
            "columnType": "DIMENSION",
            "dataType": "STRING",
            "defaultValue": "None",
            "description": "The date and time that the report was run",
            "name": "report_date"
        }
    ],
    "sqlQuery": """
        SELECT
            CUSTOM_PARAMETER('report_date'),
            advertiser,
            campaign,
            (CASE 
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 60 THEN '1 | < 1 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,conversion_event_dt) <= 600 THEN '2 | 1 - 10 MIN'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 1800 THEN '3 | 10 - 30 MIN'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 3600 THEN '4 | 30 - 60 MIN'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 7200 THEN '5 | 1 - 2 HRS'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 43200 THEN '6 | 2 - 12 HRS'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 86400 THEN '7 | 12 - 24 HRS'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 604800 THEN '8 | 1 - 7 DAYS'
                ELSE '9 | 7+ DAYS'
            END) AS time_to_conversion,
            SUM(purchases) AS purchases,
            SUM(total_purchases) AS total_brand_purchases
    FROM
        amazon_attributed_events_by_conversion_time
    GROUP BY
        1,
        2,
        3,
        4
    """
}

In [ ]:
create_workflow_request = {
    "customerId": CUSTOMER_ID,
    "workflowRequest": {
        "customerId": CUSTOMER_ID,
        "requestType": "createWorkflow",
        "workflowDefinition": workflow_definition
    }
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=invoke_workflow_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(create_workflow_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
    payload_json = json.loads(payload_json)
    executionArn = payload_json.get('executionArn', '')
    sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
        "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
    execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
    print(f"Execution URL {execution_URL}")

### Update Workflow

In [ ]:
workflow_definition = {
    "workflowId": "test-wfm-1",
    "filteredMetricsDiscriminatorColumn": "filtered",
    "sqlQuery": """
        SELECT
            advertiser,
            campaign,
            (CASE 
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 60 THEN '1 | < 1 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,conversion_event_dt) <= 600 THEN '2 | 1 - 10 MIN'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 1800 THEN '3 | 10 - 30 MIN'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 3600 THEN '4 | 30 - 60 MIN'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 7200 THEN '5 | 1 - 2 HRS'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 43200 THEN '6 | 2 - 12 HRS'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 86400 THEN '7 | 12 - 24 HRS'
                WHEN SECONDS_BETWEEN (impression_dt, conversion_event_dt) <= 604800 THEN '8 | 1 - 7 DAYS'
                ELSE '9 | 7+ DAYS'
            END) AS time_to_conversion,
            SUM(purchases) AS purchases,
            SUM(total_purchases) AS total_brand_purchases
    FROM
        amazon_attributed_events_by_conversion_time
    GROUP BY
        1,
        2,
        3
    """
}
    

In [ ]:
update_workflow_request = {
    "customerId": CUSTOMER_ID,
    "workflowRequest": {
        "customerId": CUSTOMER_ID,
        "requestType": "updateWorkflow",
        "workflowDefinition": workflow_definition
    }
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=invoke_workflow_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(update_workflow_request).encode('UTF-8'),
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
    payload_json = json.loads(payload_json)
    executionArn = payload_json.get('executionArn', '')
    sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
        "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
    execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
    print(f"Execution URL {execution_URL}")

### Get Workflow

In [ ]:
workflow_id = "test-wfm-1"

In [ ]:
get_workflow_request = {
    "customerId": CUSTOMER_ID,
    "workflowRequest": {
        "customerId": CUSTOMER_ID,
        "requestType": "getWorkflow",
        "workflowId": workflow_id
    }
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=invoke_workflow_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(get_workflow_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
    payload_json = json.loads(payload_json)
    executionArn = payload_json.get('executionArn', '')
    sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
        "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
    execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
    print(f"Execution URL {execution_URL}")


### Delete Workflow

In [ ]:
workflow_id = "test-wfm-1"

In [ ]:
delete_workflow_request = {
    "customerId": CUSTOMER_ID,
    "workflowRequest": {
        "customerId": CUSTOMER_ID,
        "requestType": "deleteWorkflow",
        "workflowId": workflow_id
    }
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=invoke_workflow_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(delete_workflow_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
    payload_json = json.loads(payload_json)
    executionArn = payload_json.get('executionArn', '')
    sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
        "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
    execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
    print(f"Execution URL {execution_URL}")

#### -----------------------

# Workflow Execution Requests

### Invoke Workflow Execution

Workflows can be executed on saved or adhoc workflows. Below is an example of executing an adhoc workflow.

In [ ]:
adhoc_execution_body = {
    "workflow": {
        "workflowId": "adhoc-test-1",
        "sqlQuery": """
            SELECT 
                advertiser, 
                campaign, 
                sum(total_cost) as spend, 
                sum(impressions) as impressions, 
                count(DISTINCT user_id) as reach 
            FROM dsp_impressions 
            GROUP BY 
                advertiser, 
                campaign
        """
    },
    "timeWindowType": "MOST_RECENT_WEEK"
}

In [ ]:
create_execution_request = {
    "customerId": CUSTOMER_ID,
    "requestType": "createExecution",
    "createExecutionRequest": adhoc_execution_body
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=invoke_workflow_executions_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(create_execution_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
    payload_json = json.loads(payload_json)
    executionArn = payload_json.get('executionArn', '')
    sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
        "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
    execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
    print(f"Execution URL {execution_URL}")

### Cancel Workflow Execution

In [ ]:
# this value can be found in the Workflow Executions table
workflow_execution_id = "7d29a430-6a8a-4dbc-9148-04bfcb58c99c"

In [ ]:
cancel_execution_request = {
    "customerId": CUSTOMER_ID,
    "requestType": "cancelExecution",
    "workflowExecutionId": workflow_execution_id
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=invoke_workflow_executions_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(cancel_execution_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    payload_json = json.loads(response.get('Payload').read().decode('UTF-8'))
    payload_json = json.loads(payload_json)
    executionArn = payload_json.get('executionArn', '')
    sm_region, sm_acct_number, sm_name, sm_execution_id = re.match(
        "arn:aws:states:([^:]*-[^:]*-[0-9]+):([0-9]{12}):execution:([^:]*):([^:]*)", executionArn).groups()
    execution_URL = f"https://{sm_region}.console.aws.amazon.com/states/home?region={sm_region}#/executions/details/{executionArn}"
    print(f"Execution URL {execution_URL}")


#### -----------------------

# Workflow Execution Schedules

### Create Workflow Execution Schedule
CRON format: https://docs.aws.amazon.com/scheduler/latest/UserGuide/schedule-types.html#cron-based

Similar to creating workflow execution requests, scheduled workflows will execute any stored payload and can be set for adhoc or saved workflows. Below is an example of setting a schedule for the saved workflow created earlier in the notebook

In [ ]:
saved_execution_body= {
    "workflowId": "test-wfm-1",
    "timeWindowStart": "FIRSTDAYOFOFFSETMONTH(-2)",
    "timeWindowEnd": "FIRSTDAYOFOFFSETMONTH(-1)",
    "timeWindowType": "EXPLICIT",
    "workflow_executed_date": "now()",
    "timeWindowTimeZone": "America/New_York",
    "requireSyntheticData": False,
    "ignoreDataGaps": "True",
    "workflowExecutionTimeoutSeconds": "86400",
    "parameterValues": {
        "report_date": "now()"
    }
}

# additional parameters needed to set the schedule
schedule_expression = 'cron(0/15 * * * ? *)'
rule_name = 'testrule1'
rule_description = 'Testing the workflow schedule creation'

In [ ]:
create_execution_schedule_request = {
    "execution_request": {
        "customerId": CUSTOMER_ID,
        "requestType": "createExecution",
        "createExecutionRequest": saved_execution_body
    },
    "schedule_expression": schedule_expression,
    "rule_name": rule_name,
    "rule_description": rule_description
}

#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=create_workflow_schedule_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(create_execution_schedule_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    rule_URL = f"https://{os.environ['REGION']}.console.aws.amazon.com/events/home?region={os.environ['REGION']}#/eventbus/default/rules/{os.environ['RULE_PREFIX']}-wfm-{rule_name}" 
    print(f"Event Rule URL {rule_URL}")

### Delete Workflow Execution Schedule

In [ ]:
delete_schedule_request = {
    'rule_name': 'testrule1'
}

In [ ]:
#############################################
# Execute request
response = lambda_client.invoke(
    FunctionName=delete_workflow_schedule_lambda_name,
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(delete_schedule_request).encode('UTF-8')
)

# Print output
response_code = response.get('ResponseMetadata', {}).get('HTTPStatusCode', 0)
if response_code in range(200, 204):
    function_url = f"https://{os.environ['REGION']}.console.aws.amazon.com/lambda/home?region={os.environ['REGION']}#/functions/{delete_workflow_schedule_lambda_name}"
    print(f"Function URL {function_url}")
else:
    print(response)

----
Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0